In [199]:
#!pip install xgboost

In [200]:
#!git clone https://github.com/brewing-datacup/reto-colombia-2020

In [201]:
#!pip install patool

In [202]:
#!pip install pyunpack
#from pyunpack import Archive
#Archive("/content/reto-colombia-2020/data/colombia-data.7z").extractall("/content/drive/My Drive/Reto/")

In [203]:
import pandas as pd
import numpy as np
import datetime
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import pickle
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sb
import math
%matplotlib inline

In [204]:
clientes_df = pd.read_csv("/content/drive/My Drive/Reto/Input1_clientes_estructura.csv", sep =';')
ventas_df = pd.read_csv("/content/drive/My Drive/Reto/Input2_clientes_venta.csv", sep =';')
base_pred= pd.read_csv("/content/drive/My Drive/Reto/Input3_clientes_test.csv", sep=";")

In [205]:
ventas_df.replace({'Marca2': r'^Marca','Cupo2': r'^Cupo','CapacidadEnvase2': r'^CapacidadEnvase'}
                  , {'Marca2': 'M', 'Cupo2':'C','CapacidadEnvase2':'CE'}, regex=True,inplace=True)

In [206]:
ventas_df['Fecha'] = pd.to_datetime(ventas_df['Año'].astype(str)+ ventas_df['Mes'].apply(lambda x: '0'+str(x) if x<=9 else str(x)) +'01')
ventas_df['Marca'] = ventas_df['Marca2']+'-'+ventas_df['Cupo2']+'-'+ventas_df['CapacidadEnvase2']
ventas_df.drop(columns=['Año','Mes','Marca2','Cupo2','CapacidadEnvase2'], inplace=True)
prueba = ventas_df

In [207]:
grouped_cliente = prueba.groupby(['Cliente','Fecha']).agg({'Volumen': ['sum'],'disc': ['sum'],'nr': ['sum'],'Marca':['nunique']}).reset_index()
grouped_cliente.columns = ['Cliente','Fecha','volumen_cliente','descuento_cliente','valor_cliente','distinct_marca_cliente']

In [208]:
grouped_cliente_marca = prueba.groupby(['Cliente','Fecha', 'Marca']).agg({'Volumen': ['sum'],'disc': ['sum'],'nr': ['sum']}).reset_index()
grouped_cliente_marca.columns = ['Cliente','Fecha','Marca','volumen_marca','descuento_marca','valor_marca']

In [209]:
grouped_cliente_marca.sort_values(['Cliente','Fecha','Marca'],inplace=True)
grouped_cliente_marca['diff_vol_marca']=grouped_cliente_marca.groupby(['Cliente','Marca'])['volumen_marca'].transform(pd.Series.diff).fillna(grouped_cliente_marca['volumen_marca'])
grouped_cliente_marca['ant_volumen_marca']=grouped_cliente_marca.groupby(['Cliente','Marca'])['volumen_marca'].shift(1)

In [210]:
prueba_2 = pd.merge(grouped_cliente, grouped_cliente_marca, left_on=['Cliente','Fecha'], right_on=['Cliente','Fecha'], how="left")

In [211]:
prueba_2['p_vol'] =prueba_2['volumen_marca']/prueba_2['volumen_cliente']
prueba_2['p_desc'] =prueba_2['descuento_marca']/prueba_2['descuento_cliente']
prueba_2['p_valor'] =prueba_2['valor_marca']/prueba_2['valor_cliente']
prueba_2['p_variacion'] = (prueba_2['diff_vol_marca']/prueba_2['ant_volumen_marca']).apply(lambda x: 0 if x==math.inf else x)
prueba_2.fillna(0,inplace=True)

In [212]:
prueba_2.drop(columns=['volumen_marca','descuento_marca','valor_marca','diff_vol_marca','ant_volumen_marca'], inplace=True)

In [213]:
prueba3= pd.pivot_table(prueba_2, values=['p_vol','p_desc','p_variacion'],
                         index=['Cliente','Fecha','volumen_cliente','descuento_cliente','valor_cliente','distinct_marca_cliente'],
                         columns=['Marca'], aggfunc=np.mean).reset_index()

In [214]:
prueba3.columns = ['_'.join(col) for col in prueba3.columns]
prueba3.rename(columns={'Cliente_': 'Cliente', 
                        'Fecha_': 'Fecha',
                        'volumen_cliente_':'volumen_cliente',
                        'descuento_cliente_':'descuento_cliente',
                        'valor_cliente_':'valor_cliente',
                        'distinct_marca_cliente_':'distinct_marca_cliente'}, inplace=True)

In [215]:
prueba3.fillna(0, inplace=True)

In [216]:
ult_3_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-05-01'))]
ult_6_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-01-01'))]
ult_12_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-07-01'))]

In [217]:
ult_3_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-06-01'))]
ult_6_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-02-01'))]
ult_12_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-08-01'))]

In [218]:
columnas = prueba3.columns.to_list()
columnas.remove('Cliente')
columnas.remove('Fecha') 

estaditicas = ['mean','median']

agg_group ={}

for i in columnas :
    key=i
    agg_group[key] = estaditicas
agg_group['Fecha'] =['nunique']

In [219]:
#ult_3_meses_test["p_variacion_M_38-C_2-CE_10"][ult_3_meses_test["Cliente"]==862]
#prueba3["p_variacion_M_38-C_2-CE_10"][prueba3["Cliente"]==862]
#prueba_2[(prueba_2["Cliente"]==862) & (prueba_2["Marca"]=="M_38-C_2-CE_10")]

In [220]:
# g=ult_3_meses_test_agg[["p_variacion_M_38-C_2-CE_10_3_mean","Cliente"]].set_index("Cliente")
# g[g.isin([np.inf])]
#k=ult_3_meses_test_agg[["p_variacion_M_38-C_2-CE_10","Cliente"]].set_index("Cliente")
#k.columns.to_series()[np.isinf(k).any()]
#[k.isin([np.inf]).any(1)]
#ult_3_meses_test_agg = ult_3_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
#ult_3_meses_test_agg

In [221]:
ult_3_meses_train_agg = ult_3_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_train_agg.columns = ['_3_'.join(col) for col in ult_3_meses_train_agg.columns]
ult_3_meses_train_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_train_agg = ult_6_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_train_agg.columns = ['_6_'.join(col) for col in ult_6_meses_train_agg.columns]
ult_6_meses_train_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_train_agg = ult_12_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_train_agg.columns = ['_12_'.join(col) for col in ult_12_meses_train_agg.columns]
ult_12_meses_train_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

In [222]:
ult_3_meses_test_agg = ult_3_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_test_agg.columns = ['_3_'.join(col) for col in ult_3_meses_test_agg.columns]
ult_3_meses_test_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_test_agg = ult_6_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_test_agg.columns = ['_6_'.join(col) for col in ult_6_meses_test_agg.columns]
ult_6_meses_test_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_test_agg = ult_12_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_test_agg.columns = ['_12_'.join(col) for col in ult_12_meses_test_agg.columns]
ult_12_meses_test_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

In [223]:
clientes_df_dummies = pd.get_dummies(clientes_df)

In [224]:
base_train = pd.merge(clientes_df_dummies, ult_12_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_train = pd.merge(base_train, ult_6_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_train = pd.merge(base_train, ult_3_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(clientes_df_dummies, ult_12_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(base_test, ult_6_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(base_test, ult_3_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test.fillna(0, inplace=True)
base_train.fillna(0, inplace=True)

In [225]:
train_filtro=base_train['Cliente'].to_list()
test_filtro=base_test['Cliente'].to_list()

In [226]:
y_train = prueba_2[prueba_2['Fecha']==pd.to_datetime('2020-08-01')]
y_train = y_train[['Cliente','Marca']]
y_train = pd.get_dummies(y_train)

columns_t = y_train.columns.to_list()
columns_t.remove('Cliente') 

estadisticas_t = ['sum']

agg_group_train ={}

for i in columns_t :
    key=i
    agg_group_train[key] = estadisticas_t
    
y_train = y_train.groupby(['Cliente']).agg(agg_group_train).reset_index()
y_train.columns = ['_'.join(col) for col in y_train.columns]
y_train.rename(columns={'Cliente_': 'Cliente'}, inplace=True)
y_train = y_train[y_train['Cliente'].isin(train_filtro)]

In [227]:
y_test = prueba_2[prueba_2['Fecha']==pd.to_datetime('2020-09-01')]
y_test = y_test[['Cliente','Marca']]
y_test = pd.get_dummies(y_test)

columns_te = y_test.columns.to_list()
columns_te.remove('Cliente') 

estadisticas_te = ['sum']

agg_group_test ={}

for i in columns_te :
    key=i
    agg_group_test[key] = estadisticas_te
    
y_test = y_test.groupby(['Cliente']).agg(agg_group_test).reset_index()
y_test.columns = ['_'.join(col) for col in y_test.columns]
y_test.rename(columns={'Cliente_': 'Cliente'}, inplace=True)
y_test = y_test[y_test['Cliente'].isin(test_filtro)]

In [228]:
train_filtro2=y_train['Cliente'].to_list()
test_filtro2=y_test['Cliente'].to_list()

In [229]:
base_train = base_train[base_train['Cliente'].isin(train_filtro2)]
base_test = base_test[base_test['Cliente'].isin(test_filtro2)]

In [230]:
y_train.set_index('Cliente' , inplace = True)
y_train.rename(columns={'Marca_M_20-C_3-CE_9_sum': 'M_20-C_3-CE_9', 
                        'Marca_M_16-C_2-CE_10_sum':'M_16-C_2-CE_10',
                        'Marca_M_9-C_3-CE_12_sum':'M_9-C_3-CE_12',
                        'Marca_M_38-C_2-CE_10_sum':'M_38-C_2-CE_10',
                        'Marca_M_39-C_2-CE_10_sum':'M_39-C_2-CE_10'
                        }, inplace=True)

In [231]:
base_train.set_index('Cliente' , inplace = True)

In [232]:
# columns =["M_20-C_3-CE_9","M_16-C_2-CE_10","M_9-C_3-CE_12","M_38-C_2-CE_10","M_39-C_2-CE_10"]

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(base_train)

# for i in columns :
#   y_base=y_train[i]
#   # create the classifier
#   data_dmatrix = xgb.DMatrix(data=X_scaled,label=y_base)
#   params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,'max_depth': 5, 'alpha': 10}
#   cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,num_boost_round=50,early_stopping_rounds=10,metrics="auc                                                       ", as_pandas=True, seed=123)
# cv_results.head()

In [233]:
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier

columns =["M_20-C_3-CE_9","M_16-C_2-CE_10","M_9-C_3-CE_12","M_38-C_2-CE_10","M_39-C_2-CE_10"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(base_train)

for i in columns :
  model=SelectFromModel(RandomForestClassifier(n_estimators = 100))
  y_base=y_train[i]
  X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_scaled, y_base, test_size=0.3, random_state=8)
  model.fit(X_train, Y_train)
  cols = model.get_support(indices=True)
  selected_feat=(base_train.iloc[:,cols].columns.tolist())
  best=model.transform(X_train)
  bestx=model.transform(X_validation)

  clf = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.1, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 150)
  # Train the model using the training sets
  eval_set=[(bestx, Y_validation)]
  clf.fit(best, Y_train,early_stopping_rounds=10, eval_metric="auc", eval_set=eval_set, verbose=True)
  #y_pred = model.predict(X_validation, ntree_limit=model.best_ntree_limit)
  fpr, tpr, thresholds = metrics.roc_curve(Y_validation, clf.predict_proba(bestx,ntree_limit=clf.best_ntree_limit)[:,1])
  auc = metrics.roc_auc_score(Y_validation, clf.predict_proba(bestx)[:,1])
  print("AUC: " + str(i)+" "+ str(auc))
  
  with open("/content/drive/My Drive/Reto/smodelxgb{}.pickle".format(i), "wb") as f:
    pickle.dump(clf, f) 
    pickle.dump(selected_feat, f)


[0]	validation_0-auc:0.713594
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.730668
[2]	validation_0-auc:0.735879
[3]	validation_0-auc:0.746472
[4]	validation_0-auc:0.751537
[5]	validation_0-auc:0.75199
[6]	validation_0-auc:0.752134
[7]	validation_0-auc:0.750966
[8]	validation_0-auc:0.764115
[9]	validation_0-auc:0.765221
[10]	validation_0-auc:0.765068
[11]	validation_0-auc:0.764503
[12]	validation_0-auc:0.76512
[13]	validation_0-auc:0.769566
[14]	validation_0-auc:0.77142
[15]	validation_0-auc:0.771312
[16]	validation_0-auc:0.772899
[17]	validation_0-auc:0.775494
[18]	validation_0-auc:0.77612
[19]	validation_0-auc:0.776976
[20]	validation_0-auc:0.777364
[21]	validation_0-auc:0.779617
[22]	validation_0-auc:0.780878
[23]	validation_0-auc:0.781955
[24]	validation_0-auc:0.78284
[25]	validation_0-auc:0.783851
[26]	validation_0-auc:0.784159
[27]	validation_0-auc:0.783989
[28]	validation_0-auc:0.784469
[29]	validation_0-auc:0.785013
[30]	validation_0-auc

In [234]:
# LOAD MODEL
y_test.set_index('Cliente' , inplace = True)
base_test.set_index('Cliente' , inplace = True)
y_test.rename(columns={'Marca_M_20-C_3-CE_9_sum': 'M_20-C_3-CE_9', 
                        'Marca_M_16-C_2-CE_10_sum':'M_16-C_2-CE_10',
                        'Marca_M_9-C_3-CE_12_sum':'M_9-C_3-CE_12',
                        'Marca_M_38-C_2-CE_10_sum':'M_38-C_2-CE_10',
                        'Marca_M_39-C_2-CE_10_sum':'M_39-C_2-CE_10'
                        }, inplace=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(base_test)


for i in columns: 
   with open("/content/drive/My Drive/Reto/smodelxgb{}.pickle".format(i), "rb") as f:
     model = pickle.load(f)
     selected_feat = pickle.load(f)
     base=base_test[selected_feat]
     X_scaled = scaler.fit_transform(base)
     predicted= model.predict(X_scaled)
     #print(model.best_iteration)
     fpr, tpr, thresholds = metrics.roc_curve(y_test[i], model.predict_proba(X_scaled,ntree_limit=model.best_ntree_limit)[:,1])
     auc = roc_auc_score(y_test[i], model.predict_proba(X_scaled)[:,1])
     print("AUC: " + str(i)+" "+ str(auc))

AUC: M_20-C_3-CE_9 0.7824642646837824
AUC: M_16-C_2-CE_10 0.5045026568718773
AUC: M_9-C_3-CE_12 0.8419731667378887
AUC: M_38-C_2-CE_10 0.8328962677167587
AUC: M_39-C_2-CE_10 0.8316414064121563


In [235]:
base_filtro=base_pred['Cliente'].to_list()
prueba4 = prueba3[prueba3['Cliente'].isin(base_filtro)]
clientes_df_dummies_2= clientes_df_dummies[clientes_df_dummies["Cliente"].isin(base_filtro)]

In [236]:
ult_3_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-07-01'))]
ult_6_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-03-01'))]
ult_12_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-09-01'))]

ult_3_meses_base_agg = ult_3_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_base_agg.columns = ['_3_'.join(col) for col in ult_3_meses_base_agg.columns]
ult_3_meses_base_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_base_agg = ult_6_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_base_agg.columns = ['_6_'.join(col) for col in ult_6_meses_base_agg.columns]
ult_6_meses_base_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_base_agg = ult_12_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_base_agg.columns = ['_12_'.join(col) for col in ult_12_meses_base_agg.columns]
ult_12_meses_base_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [237]:
base_fin = pd.merge(clientes_df_dummies_2,ult_12_meses_base_agg,left_on=['Cliente'], right_on=['Cliente'], how="left")
base_fin= pd.merge(base_fin, ult_6_meses_base_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_fin= pd.merge(base_fin, ult_3_meses_base_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_fin.set_index('Cliente' , inplace = True)

In [238]:
base_fin.fillna(0, inplace=True)
#base_fin1 =base_fin[base_fin.isin([np.inf]).any(1)]
#base_fin1
base_fin.columns.to_series()[np.isinf(base_fin).any()]

Series([], dtype: object)

In [239]:
scaler = StandardScaler()
X_scaled1 = scaler.fit_transform(base_fin)


for i in columns: 
   with open("/content/drive/My Drive/Reto/smodelxgb{}.pickle".format(i), "rb") as f:
     model = pickle.load(f)
     selected_feat = pickle.load(f)

     base=base_fin[selected_feat]
     X_scaled1 = scaler.fit_transform(base)
     base_fin[i]=np.round(model.predict_proba(X_scaled1,ntree_limit=model.best_ntree_limit)[:,1],3)#df['column'].round(1)
base_predic=base_fin[columns]

In [240]:
#base_predic.reset_index(inplace=True)
base_predic.columns = ['Marca1',	'Marca2',	'Marca3',	'Marca_Inno1',	'Marca_Inno2']


In [241]:
base_predic.to_csv("/content/drive/My Drive/Reto/outputxgb.csv")

In [242]:
base_predic

Marca1  Marca2  Marca3  Marca_Inno1  Marca_Inno2
Cliente                                                  
1         0.087   0.304   0.075        0.395        0.533
2         0.420   0.304   0.081        0.145        0.090
3         0.129   0.304   0.084        0.242        0.318
4         0.491   0.308   0.122        0.305        0.156
5         0.400   0.304   0.254        0.118        0.081
...         ...     ...     ...          ...          ...
20411     0.066   0.304   0.056        0.929        0.914
20414     0.083   0.304   0.075        0.759        0.746
20434     0.078   0.304   0.068        0.879        0.874
20462     0.125   0.304   0.067        0.962        0.948
20569     0.059   0.304   0.085        0.758        0.625

[9302 rows x 5 columns]